In [62]:
import numpy as np
import cv2
import time 


In [63]:
pwd


'C:\\Users\\User'

In [64]:
image_BGR = cv2.imread('test41.jpg')


In [65]:
cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)
cv2.imshow('Original Image', image_BGR)
cv2.waitKey(0)
cv2.destroyWindow('Original Image')
h, w = image_BGR.shape[:2] 

In [66]:
blob = cv2.dnn.blobFromImage(image_BGR, 1 / 255.0, (416, 416),
                             swapRB=True, crop=False)

In [67]:
with open('coco.names') as f:
    labels = [line.strip() for line in f]


network = cv2.dnn.readNetFromDarknet('yolov3.cfg',
                                     'yolov3.weights')


layers_names_all = network.getLayerNames()

layers_names_output = \
    [layers_names_all[i[0] - 1] for i in network.getUnconnectedOutLayers()]


In [68]:
probability_minimum = 0.5
threshold = 0.3
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

In [69]:
network.setInput(blob)  # setting blob as input to the network
start = time.time()
output_from_network = network.forward(layers_names_output)
end = time.time()

In [70]:
print('Objects Detection took {:.5f} seconds'.format(end - start))
bounding_boxes = []
confidences = []
class_numbers = []

Objects Detection took 4.60343 seconds


In [71]:
for result in output_from_network:   
    for detected_objects in result:
        
        scores = detected_objects[5:]       
        class_current = np.argmax(scores)        
        confidence_current = scores[class_current]

        

        
        if confidence_current > probability_minimum:                                                        
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            x_center, y_center, box_width, box_height = box_current
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)





results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                           probability_minimum, threshold)
print("results:",results)

results: [[ 7]
 [ 2]
 [ 1]
 [ 6]
 [ 9]
 [ 8]
 [ 3]
 [ 4]
 [12]
 [13]
 [10]]


In [72]:
counter = 1

if len(results) > 0:
    for i in results.flatten():
        print('Object {0}: {1}'.format(counter, labels[int(class_numbers[i])]))

        counter += 1

   
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

        colour_box_current = colours[class_numbers[i]].tolist()
        
        cv2.rectangle(image_BGR, (x_min, y_min),
                      (x_min + box_width, y_min + box_height),
                      colour_box_current, 2)
        text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                               confidences[i])

        cv2.putText(image_BGR, text_box_current, (x_min, y_min - 5),
                    cv2.FONT_HERSHEY_COMPLEX, 0.7, colour_box_current, 2)


Object 1: car
Object 2: car
Object 3: car
Object 4: car
Object 5: car
Object 6: person
Object 7: person
Object 8: person
Object 9: person
Object 10: person
Object 11: person


In [73]:
print()
print('Total objects been detected:', len(bounding_boxes))
print('Number of objects left after non-maximum suppression:', counter - 1)


cv2.namedWindow('Detections', cv2.WINDOW_NORMAL)
cv2.imshow('Detections', image_BGR)
cv2.waitKey(0)
cv2.destroyWindow('Detections')


Total objects been detected: 14
Number of objects left after non-maximum suppression: 11


In [80]:
cv2.imwrite("test42.jpg",image_BGR)

True

In [84]:
i = cv2.imread('test42.jpg')
cv2.imshow("jhgr",i)